In [0]:
import random

- 概况  
  1. 通常是实际应用排序中最好的选择，因为其平均性能非常好
    - 期望的时间复杂度为 $\Theta(nlg(n))$，而且 $\Theta(nlg(n))$ 中隐含的常数因子非常小
  2. 能够进行原址排序，可以在虚拟环境中很好的工作
    - 如果输入数组中仅有**常数个元素**需要在排序过程中存储在数组之外，则称排序算法是**原址**的  

## 7.1 快速排序的描述

- 采用分治思想，包括分解，解决和合并

### 数组划分 -- PARTITION  
- 实现对子数组组 $A[p \cdots r]$ 的原址重排

#### 代码实现

In [0]:
def partition(A, p, r):
  x = A[r]  # 主元
  i = p - 1
  for j in range(p, r):
    if A[j] <= x:
      i += 1
      A[i], A[j] = A[j], A[i]
  A[i+1], A[r] = A[r], A[i+1]
  return i + 1

In [7]:
A = [2, 8, 7, 1, 3, 5, 6, 4]
print(partition(A, 0, len(A)-1))
print(A)

3
[2, 1, 3, 4, 7, 5, 6, 8]


#### 执行过程分析

<img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20191215160554.png width=600>

#### 快速排序

In [0]:
def quicksort(A, p, r):
  if p < r:
    q = partition(A, p, r)
    quicksort(A, p, q-1)
    quicksort(A, q+1, r)

In [10]:
A = [2, 8, 7, 1, 3, 5, 6, 4]
quicksort(A, 0, len(A)-1)
print(A)

[1, 2, 3, 4, 5, 6, 7, 8]


### 练习

#### 7.1-2

当 $A[p \cdots r]$ 中的元素均相同时，返回的 $q$ 值为 $r$

In [14]:
A = [1 for i in range(10)]
print(partition(A, 0, 9))

9


可按如下方法修改 PARTITION, 使其返回值为 
$ \lfloor (p+r)/2 \rfloor$

In [0]:
def partition_handle_same_value(A, p, r):
  x = A[r]
  i = p - 1
  for j in range(p, r):
    if A[j] <= x:
      if A[j] == x and (j%2) == (p)%2:
        continue
      i += 1
      A[i], A[j] = A[j], A[i]
  A[i], A[r] = A[r], A[i]
  return i + 1

In [26]:
A = [1 for i in range(11)]
print(partition_handle_same_value(A, 0, 9))
print(partition_handle_same_value(A, 0, 10))

4
5


#### 7.1-4  
以非递增序排序

In [0]:
def partition_reverse(A, p, r):
  x = A[r]
  i = p - 1
  for j in range(p, r):
    if A[j] >= x:
      i += 1
      A[i], A[j] = A[j], A[i]
  A[i+1], A[r] = A[r], A[i+1]
  return i + 1

In [0]:
def quicksort_reverse(A, p, r):
  if p < r:
    q = partition_reverse(A, p, r)
    quicksort_reverse(A, p, q-1)
    quicksort_reverse(A, q+1, r)

In [33]:
A = [2, 8, 7, 1, 3, 5, 6, 4]
quicksort_reverse(A, 0, len(A)-1)
print(A)

[8, 7, 6, 5, 4, 3, 2, 1]


## 7.2 快速排序的性能

### 最坏情况划分

- 当划分产生的两个子问题分别包括 $n-1$ 个元素 和 $0$ 个元素时，即为快速排序的是坏情况（证明在 7.4.1 节）
- 假设算法的每一次递归调用都出现了这种最坏情况，则可得递推式：
$$T\left( n \right) = T\left( {n - 1} \right) + T\left( 0 \right) + \Theta \left( 1 \right) = T\left( {n - 1} \right) + \Theta \left( n \right)$$
结果为 $\Theta(n^2)$
- 当数组已经完全有序时，并且主元选为最后一个元素时，即会出现此种情况

### 最好情况划分

- 最好情况划分，即为最平衡的划分，此时一个子问题的规模为 $T( \lfloor n/2 \rfloor)$, 另一个问题的规模为
$T(\lceil n/2 \rceil - 1)$
- 此时，算法运行时间递推式为
$$ T(n) = 2T(n/2) + \Theta(n) $$
由主定理可得， $T(n) = \Theta(nlg(n))$

### 平衡的划分

- 快速排序的平均运行时间更加接近于其最好情况，而非最坏情况
- 考虑 $9:1$ 的不平衡划分，可得递推式
$$ T(n) = T(9n/10) + T(n/10) + \Theta(n) $$
- 通过递推树法进行分析  
<img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20191215170107.png width=600>  
可得其运行时间为 $O(nlg(n))$
- 任何一种常数比例的划分都会产生深度为 $\Theta(lg(n))$ 的递推树，其中每一层的时间代价都是 $O(n)$
  - 因此，只要划分是常数比例的，算法的运行时间总是 $O(nlg(n))$

### 对于平均情况的直接观察

- 快速排序的行为依赖于输入数组中元素的值的相对顺序，而不是某些特定值本身，因此需要假设输入数据的所有排列都是等概率的
- 当好和差的划分交替出现时，快速排序的时间复杂度与全是好的划分时是一样的，仍然是 $O(nlg(n))$。区别是 $O$ 符号
中隐含的**常数因子项**略大一些    
  - 将差的划分产生的代价归并入划分步骤中
  - 示意图  
  <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20191215171848.png width=600>

## 7.3 快速排序的随机化版本

- 随机化版本的快速排序常作为大数据输入情况下的排序算法
- 可以采用**随机抽样**的随机化技术来实现随机化版本的快速排序
  - PARTITION 时随机的选择主元

### RANDOMIZED-PARTITION

In [0]:
def randomized_partition(A, p, r):
  i = 